# Gaussian Filter
Want to explore using a Gaussian filter over the input before feeding it to the network. This notebook just needs to demonstrate the effect of a gaussian filter on a small section of data.

Guasian filter with `sigma=0.2` has a mild smoothing effect on the noisy regions whilst mainintaing mostly the same geometry in the higher density regions. The optimal intuitively seems to be around `sigma=0.2-0.4` before a noticeable loss of detail sets in.

Below we visualise `sigma=[0, 0.2,.3, 0.4, 0.5, .6, .7 ]`:
![Visualisation comaparisons](GuassianFilter_Testing.png)


## 1. Load data using standard dataloaders

In [1]:
from data_utils.S3DISDataLoader import S3DISDataset
import data_utils.MastersDataLoader as MastersDataLoader
import numpy as np
import torch
import time

from types import SimpleNamespace

root = 'data/s3dis/songomnara'
NUM_POINT = 2048
NUM_CLASSES = 2
BATCH_SIZE = 1
args = {'test_area': 2, 'block_size': 1,
        'test_sample_rate': 1, 'shuffle_training_data': True}
args = SimpleNamespace(**args)

print("start loading training data ...")
TRAIN_DATASET = S3DISDataset(split='train', data_root=root, num_point=NUM_POINT, test_area=args.test_area,
                             block_size=args.block_size, sample_rate=1.0, transform=None, num_classes=NUM_CLASSES)
trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE,
                                              shuffle=args.shuffle_training_data, num_workers=0,
                                              pin_memory=True, drop_last=True,
                                              worker_init_fn=lambda x: np.random.seed(x + int(time.time())))

  0%|          | 0/8 [00:00<?, ?it/s]

start loading training data ...


100%|██████████| 8/8 [00:01<00:00,  4.71it/s]

Labelweights=[1.        1.3157682]
Totally 9572 samples in train set.


In [2]:
data_iter = iter(trainDataLoader)
points, target, room_idx = next(data_iter)  # points (1,NUM_POINTxbatch_size,9), target(1,NUM_POINTxbatch_size)

## 2. Define the filter

In [3]:
def gaussian_filter(points, sigma=1):
    """Applies a gausssian filter over a set of 3D points
    """
    from scipy.ndimage import gaussian_filter
    points = gaussian_filter(points, sigma=sigma)
    
    return points

## 3. Visualise

In [4]:
import open3d
from Visualisation_utils import convert_class_to_turborgb255


print("Show initial points")
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(points[0,:,:3])
pcd.colors = open3d.utility.Vector3dVector(np.array([convert_class_to_turborgb255(i, 1) for i in target[0,:]])/255)
# pcd.colors = open3d.utility.Vector3dVector(points[0,:,3:6])
# pcd.colors = open3d.utility.Vector3dVector(convert_class_to_turborgb255)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-11-19 14:58:15,311 - utils - NumExpr defaulting to 4 threads.


Show initial points


In [5]:
open3d.visualization.draw_geometries([pcd])

In [6]:
print("Apply gaussian filter over the inputs")
filtered_points = gaussian_filter(points[0,:,:3])

pcd_filtered = open3d.geometry.PointCloud()
pcd_filtered.points = open3d.utility.Vector3dVector(filtered_points[:,:3]+np.array([3,0,0]))
pcd_filtered.colors = open3d.utility.Vector3dVector(np.array([convert_class_to_turborgb255(i, 1) for i in target[0,:]])/255)
# pcd.colors = open3d.utility.Vector3dVector(convert_class_to_turborgb255)

Apply gaussian filter over the inputs


In [7]:
open3d.visualization.draw_geometries([pcd_filtered, pcd])

## Visualise the smoothing effect

In [8]:
pcds = []
for i, sigma in enumerate([0, 0.1, 0.2, 0.4, 0.5]):
    pcds.append(open3d.geometry.PointCloud())
    pcds[-1].points = open3d.utility.Vector3dVector(
        gaussian_filter(points[0, :, :3], sigma)[:, :3]+np.array([3*i, 0, 0]))
    pcds[-1].colors = open3d.utility.Vector3dVector(
        np.array([convert_class_to_turborgb255(i, 1) for i in target[0, :]])/255)

open3d.visualization.draw_geometries(pcds)


## Visualising the smoothing effect with comparison to the original

In [9]:
pcds = []
for i, sigma in enumerate([0, 0.2,.3, 0.4, 0.5, .6, .7 ]):
    pcds.append(open3d.geometry.PointCloud())
    pcds[-1].points = open3d.utility.Vector3dVector(
        np.vstack((np.vstack((gaussian_filter(points[0, :, :3], sigma)[:, :3], np.asarray(pcd.points)))+np.array([3*i, 0, 0]), gaussian_filter(points[0, :, :3], sigma)[:, :3]+np.array([3*i, 0,-7]))))
    pcds[-1].colors = open3d.utility.Vector3dVector(
        np.vstack((np.array([convert_class_to_turborgb255(i, 1) for i in target[0, :]])/255, np.array([convert_class_to_turborgb255(i, 1) for i in target[0, :]])/255*2, np.array([convert_class_to_turborgb255(i, 1) for i in target[0, :]])/255)))

open3d.visualization.draw_geometries(pcds)
